<a href="https://colab.research.google.com/github/shuaigezhu/starDist/blob/master/RandomForestStar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from math import pi
from random import randint
from torch import nn
from torch.distributions.kl import kl_divergence
from sklearn import preprocessing

#!unzip -q "/content/gaia_pos.zip"
# Get data file names
path =r'/content/gaia_pos'
filenames = glob.glob(path + "/*.csv")

dfs = []
error_dfs = []


def Normalization(data):
  data_f = data.astype(float)
  data_mean = np.mean(data_f, axis=0, keepdims=True)
  data_n = data_f - data_mean
  data_range = np.max(np.abs(data_n), axis=0, keepdims=True)
  data_n = data_n / data_range
  
  return data_n

# Select specific colomns which are "ra", "dec", "magnitude", "wavelength" and "parallax" and "parallax error"

for filename in filenames:
  data = pd.read_csv(filename, header=None)
  select_data = data.iloc[1:,[2, 4, 10, 11, 6]]
  parallax_error = data.iloc[1:,[7]]
  
  dfs.append(select_data)
  error_dfs.append(parallax_error)

# Combine the three files of the different type of filter into one pandas framework

frame = pd.concat(dfs, axis=0, ignore_index=True, sort = False)
parallax_error_frame = pd.concat(error_dfs, axis=0, ignore_index=True, sort = False)

frame = frame.rename(columns={2: "ra", 4: "dec", 10: "magnitude", 11: "wavelength", 6: "parallax"})
parallax_error_frame = parallax_error_frame.rename(columns={7:"parallax error"})
#print(parallax_error_frame.head())

# Split the dataset into training and testing part.
ratio = 0.9 #the ratio is train/all 
msk = np.random.rand(len(frame)) < ratio
train_frame = frame[msk]
test_frame = frame[~msk]
parallax_error_test_frame = parallax_error_frame[~msk]

def split_into_xy(dataframe):  
  x = dataframe.iloc[:, [0,1,2,3]]
  y = dataframe.iloc[:, [4]]
  
  return x , y

train_x, train_y = split_into_xy(train_frame)
test_x, test_y = split_into_xy(test_frame)

#convert into numpy array and type of float
train_x = np.array(train_x)
train_x = train_x.astype(float)
train_y = np.array(train_y)
train_y = train_y.astype(float)

test_x = np.array(test_x)
test_x = test_x.astype(float)
test_y = np.array(test_y)
test_y = test_y.astype(float)

test_parallax_error = np.array(parallax_error_test_frame).astype(float)

r_scaler = preprocessing.RobustScaler()
train_norm_x = Normalization(train_x)
train_norm_y = train_y
#train_norm_y = r_scaler.fit_transform(train_y)
test_norm_x = Normalization(test_x)
test_norm_y = test_y
#test_norm_y = r_scaler.fit_transform(test_y)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(train_norm_x.shape)

(819681, 4)


### **random forest**

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

locations = np.random.choice(train_norm_x.shape[0],
                                 size=100000,
                                 replace=False)

X = train_norm_x[locations,:]
Y = train_norm_y[locations,:]
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)# Train the model on training data
rf.fit(X, np.reshape(Y,Y.shape[0])) 


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [0]:
predictions = rf.predict(test_norm_x)
tmp = np.reshape(test_norm_y, test_norm_y.shape[0])
print (np.median(1-(abs(tmp - predictions)/predictions)))

print(predictions)
print(tmp)


0.457948723414777
[1.33217432 0.8397331  1.42876108 ... 1.46013703 1.47429116 1.39596781]
[2.39696324 0.85392091 0.43093035 ... 0.19094035 0.28485553 0.38405779]


In [29]:

print(test_norm_x[:,2])
print(test_norm_y)


[-0.14875511  0.07114928 -0.19000589 ... -0.04255316 -0.06606188
  0.17827017]
[[2.39696324]
 [0.85392091]
 [0.43093035]
 ...
 [0.19094035]
 [0.28485553]
 [0.38405779]]


### **Linear Regression**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#locations = np.random.choice(train_norm_x.shape[0],
 #                                size=100000,
  #                               replace=False)

#X = train_norm_x[locations,:]
#Y = train_norm_y[locations,:]
X = train_norm_x
Y = train_norm_y
Y_test = np.reshape(test_norm_y, test_norm_y.shape[0])
X_test = test_norm_x
lm = linear_model.LinearRegression()
model = lm.fit(X, Y)
predictions = lm.predict(X_test)
print("Size of dataframe: " + str(X.shape[0]))
print("Polynomial regression:")
print("\tDegree = 1,  Score = " + str(model.score(X_test, Y_test)))
degree_max = 5

for d in range(2, degree_max + 1):
    print("\tDegree = " + str(d), end="")
    polynomial_features = PolynomialFeatures(degree=d, interaction_only=False)
    linear_regression = LinearRegression()
    X_poly = polynomial_features.fit_transform(X)
    model=linear_regression.fit(X_poly, Y)
    X_test_poly = polynomial_features.fit_transform(X_test)
    Y_test_pred = linear_regression.predict(X_test_poly)
    print(",  Score = " + str(model.score(X_test_poly, Y_test)))

Size of dataframe: 819681
Polynomial regression:
	Degree = 1,  Score = 0.022743585395110122
	Degree = 2,  Score = 0.06458923811743333
	Degree = 3,  Score = -10.515556709259242
	Degree = 4,  Score = -1.3331380207557933e+19
	Degree = 5,  Score = -1.2725886278644527e+19
